<a href="https://colab.research.google.com/github/A-Belknap/austin_INFO5707_Spring2022/blob/main/lab_assignment_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The sixth Lab-assignment (03/24/2022, 50 points in total)

The purpose of this exercise is to build a simple predicition model which can helpyou understand the workflow of machine learning. 

### Task Decription 

The goal of this assignment is to predict bike share use, given the hour, day, and information about the weather. Companies like Divvy try to predict how much demand there will be for bikes on any given day to allocate resources to redistribute bikes so that, ideally, very few bike stations are ever full (when you can’t park your bike) or empty (when you can’t pick up a bike if you want to).

The data (link: https://github.com/unt-iialab/info5502-spring2022/tree/main/datasets/lab_assignment_06) in Github provides detailed information on the data set and necessary downloads.

### Data Description

You are provided hourly rental data spanning two years (link: https://github.com/unt-iialab/info5502-spring2022/tree/main/datasets/lab_assignment_06). For this task, the training set is comprised of the first 16 days of each month, while the test set is the 17-19th day of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period. That is, predict "count" without using "count" or its components "casual" and "registered".

Data Fields

datetime - hourly date + timestamp

season - 1 = spring, 2 = summer, 3 = fall, 4 = winter

holiday - whether the day is considered a holiday

workingday - whether the day is neither a weekend nor holiday

weather -

1 - Clear, Few clouds, Partly cloudy, Partly cloudy

2 - Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist

3 - Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

4 - Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

temp - temperature in Celsius

atemp - "feels like" temperature in Celsius

humidity - relative humidity

windspeed - wind speed

casual - number of non-registered user rentals initiated

registered - number of registered user rentals initiated

count - number of total rentals

### Output

Submission Format

Your output (a separate file) must have a header line and should be structured in the following format:

 datetime,count
 
 2011-01-20 00:00:00,0 
 
 2011-01-20 01:00:00,0
 
 2011-01-20 02:00:00,0
 
 ...
 
The tutorial code should demonstrate how to generate such a file from a very simple prediction model. Note, these prediction are to be done on the test file under the data tab, where you do not know the actual count, and should match the rows of the test file in count and order.

Your predictions should be compared to the ground truth information (sample_prediction.csv). Score are calculated using Root Mean Squared Error (RMSE).

### Tips

●	Add features: Pick columns/features from the data you already have. Or make a new feature from the ones you have. For example, the tutorial makes ‘hour’ from the datetime stamp, which seemed very useful. How about ‘month’?

●	Model selection: Try different models. Make sure they are regression models rather than classification models. Tip: random forest regression is a good, all around modeling strategy on complicated data sets.

●	Model tuning: Almost all regression models have parameters to tune (“hyperparameters”). E.g. polynomial regression has the degree of the polynomial (n = 1 for a line, n=2 for a quadratic fit, n=3 for a cubic fit…). Generally, one extreme makes the model too simple (e.g. a line for a curved set of points) and the other extreme makes the model overfit/be too complex, and usually the right choice is in between. For some models it is obvious what to tune (e.g. k for k nearest neighbors regression) and some don’t need much tuning with defaults that often work well. e.g. try changing the number of trees used in the random forest model!

●	Cross validation: The tutorial has a simple way of separating training and test data, however, there are better ways of splitting training and test data. Look into cross validation techniques, which are more reliable than an arbitrary split of training and test data.

●	Separate models for ...: Notice that count comes from just adding casual riders and registered riders. However, what if these two types of riders acted very differently? It might make sense to make two separate models and just add the results of both models together. This is also true for any subsets of your data that may behave wildly differently (e.g. create a separate model for each season?)

### Your code

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
### You code here:

import pandas as pd
import numpy as np
import csv as csv
from datetime import datetime
import matplotlib.pyplot as plt

%matplotlib inline

# read the training data
train_filepath = "/content/drive/MyDrive/Lab 6/train_luc.csv"
train_df = pd.read_csv(train_filepath, header=0)

print("\nNumber of entries:",train_df.shape[0],"and number of features:",train_df.shape[1],"\n")
train_df.head()

#
train_df.describe()




Number of entries: 9174 and number of features: 12 



,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,9174.000000,9174.000000,9174.000000,9174.000000,9174.000000,9174.000000,9174.000000,9174.000000,9174.000000,9174.000000,9174.000000
mean,2.505559,0.031284,0.678875,1.414868,20.130401,23.578433,61.715064,12.737931,35.713647,154.868106,190.581753
std,1.116618,0.174094,0.466934,0.635363,7.940504,8.617957,19.401829,8.199027,49.667738,150.981155,181.011530
min,1.000000,0.000000,0.000000,1.000000,0.820000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.940000,16.665000,46.000000,7.001500,4.000000,35.000000,41.000000
50%,3.000000,0.000000,1.000000,1.000000,20.500000,24.240000,61.000000,11.001400,16.000000,117.000000,144.000000
75%,4.000000,0.000000,1.000000,2.000000,27.060000,31.060000,78.000000,16.997900,48.000000,222.000000,282.000000
max,4.000000,1.000000,1.000000,4.000000,41.000000,45.455000,100.000000,56.996900,362.000000,886.000000,977.000000


In [9]:
# generate new column with just the hour of the bike rental
def hour_of_day(dt):
    return datetime.strptime(dt, "%Y-%m-%d %H:%M:%S").time().hour
train_df['hour'] = train_df['datetime'].map(hour_of_day)
train_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,4


In [10]:
# drop the datetime column becuase it isn't needed anymore
train_df = train_df.drop(columns='datetime')
# drop the causal and registered columns because we aren't going to
# to be separating them into different datasets
train_df = train_df.drop(columns='casual')
train_df = train_df.drop(columns='registered')
train_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour
0,1,0,0,1,9.84,14.395,81,0.0,16,0
1,1,0,0,1,9.02,13.635,80,0.0,40,1
2,1,0,0,1,9.02,13.635,80,0.0,32,2
3,1,0,0,1,9.84,14.395,75,0.0,13,3
4,1,0,0,1,9.84,14.395,75,0.0,1,4


In [11]:
# reorder the columns so that count is at the end as the predicted variable
new_cols = ['hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'count']
train_df = train_df[new_cols]
print(train_df)


      hour  season  holiday  workingday  weather   temp   atemp  humidity  \
0        0       1        0           0        1   9.84  14.395        81   
1        1       1        0           0        1   9.02  13.635        80   
2        2       1        0           0        1   9.02  13.635        80   
3        3       1        0           0        1   9.84  14.395        75   
4        4       1        0           0        1   9.84  14.395        75   
...    ...     ...      ...         ...      ...    ...     ...       ...   
9169    19       4        0           0        1  14.76  17.425        93   
9170    20       4        0           0        2  15.58  19.695        82   
9171    21       4        0           0        2  14.76  18.940        93   
9172    22       4        0           0        2  16.40  20.455        82   
9173    23       4        0           0        2  14.76  17.425        93   

      windspeed  count  
0        0.0000     16  
1        0.0000     40  


In [12]:
# use ANOVA Tto determnine which features to use
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
array = train_df.values
X = array[:,0:9]
Y = array[:,9]

# feature extraction
test = SelectKBest(score_func=f_classif, k=3)
fit = test.fit(X, Y)
# summarize scores
set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)
# summarize selected features
print(features[0:6,:])


[7.093 1.54  0.869 1.18  1.002 3.781 3.751 2.674 1.275]
[[ 0.     9.84  14.395]
 [ 1.     9.02  13.635]
 [ 2.     9.02  13.635]
 [ 3.     9.84  14.395]
 [ 4.     9.84  14.395]
 [ 5.     9.84  12.88 ]]


In [13]:
# generate model using selected features
cols = ['hour', 'temp', 'atemp']

# import multiples models to make testing differents models easier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor

# random forest regression was chosen because it works well 
# withg complicated datasets
# 10 was chosen as the number of estimators becasue it is a standard number of
# estimators
#model = DecisionTreeRegressor()
#model = LinearRegression()
#model = KNeighborsRegressor(n_neighbors = 5)
#model = LinearSVR()
model = RandomForestRegressor(n_estimators = 10)

print("columns selected for later:",cols)

columns selected for later: ['hour', 'temp', 'atemp']


In [14]:
# split the data into a training set and test set
from sklearn.model_selection import train_test_split

train, test = train_test_split(train_df, test_size=0.25, random_state=42, shuffle=True)

new_train_df = train
new_test_df = test

print("samples in the new training subset:",len(new_train_df))
print("samples in the new test subset:",len(new_test_df))

new_train_df.head()


samples in the new training subset: 6880
samples in the new test subset: 2294


,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
7224,17,3,0,0,1,32.80,38.635,59,32.9975,436
3063,23,3,0,1,1,26.24,30.305,65,16.9979,126
2223,23,2,0,1,1,22.96,26.515,68,15.0013,110
100,9,1,0,1,1,9.02,9.850,37,22.0028,115
2319,23,3,0,0,1,29.52,33.335,54,19.0012,154


In [15]:
# fit the model to the split training data
model.fit(new_train_df[cols], new_train_df['count'])

# predict on the split test data
pred_count = model.predict(new_test_df[cols])

# score the model on the split test data
from sklearn.metrics import mean_squared_error
rms = np.sqrt(mean_squared_error(new_test_df['count'],pred_count))
print("RMS error:",rms)

RMS error: 121.10145825848973


In [16]:
# read in the final test data
test_df = pd.read_csv('/content/drive/MyDrive/Lab 6/test_luc.csv', header=0)
print("\nNumber of samples:",test_df.shape[0] ,"and number of features:",test_df.shape[1],"\n")

# add the hour feature to the final test dataset
test_df['hour'] = test_df['datetime'].map(hour_of_day)

# show the test data output to be sure it read in correctly and added the column
test_df.head()

# fit the selected model to the original dataset
model.fit( train_df[cols], train_df['count'])

# apply to the final test dataset
pred_count = model.predict(test_df[cols])

# add the the column of predicted counts
test_df['count'] = pred_count

# save the predicted counts as a csv
test_df = test_df[['datetime','count']].to_csv('/content/drive/MyDrive/Lab 6/my_prediction.csv', 
    index=False, header=True)
print("Prediction complete. Saved as my_prediction.csv")


Number of samples: 1712 and number of features: 9 

Prediction complete. Saved as my_prediction.csv
